In [3]:
#concise single run version
import pandas as pd
import numpy as np
import requests #used to perform gets
import json #we are storing intially the files as JSON from our gets
import string
import random
import boto3 as b3
import sys

def insert_names(arr):
    insert_string = ''
    url_multiple_names = 'https://api.nationalize.io?'
    for name in arr:
        insert_string = insert_string + 'name[]=' + name + '&'
    insert_string = insert_string[:-1]
    new_url = url_multiple_names + insert_string
    return new_url

def get_json(arr_1):
    names_1    = insert_names(arr_1)
    names_url  = insert_names(names_1)
    response_1 = requests.get(names_url)
    data_1 = response_1.json()
    return data_1
    
def fix_df_country(df):
    data_1 = get_json(df)
    new_list_all_data = []
    data_1 = pd.DataFrame(data_1)
    #iterate through every spot
    for i in range(len(data_1)):
        i = i + 0
        for spot in data_1.iloc[i]['country']:
            for blank in data_1.iloc[i]['country']:
                spot_country = blank['country_id']
                spot_prop    = blank['probability']
                new_list_all_data.append([data_1.iloc[i]['name'], spot_country, spot_prop])
    clean_df = pd.DataFrame(new_list_all_data)
    return clean_df

def id_generator(size=10, chars=string.ascii_uppercase + string.digits):
    return ''.join(random.choice(chars) for _ in range(size))

def csver(names):
    names_df = fix_df_country(names)
    names_df.to_csv('spotname')

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


In [2]:
names = ['pedro','austin', 'juan']

In [27]:
csver(sys.argv)


In [ ]:
csver(names)

In [4]:
client2 = b3.client('s3')

In [5]:
client2.create_bucket(Bucket = 'yetanothercorrupteds3')

{'ResponseMetadata': {'RequestId': 'DQVF0P1VNQEFFWJJ',
  'HostId': '8ducK1ADT1QxHYB2tGy7ce0F85v/f7e5Jh+0WoA55Exoj4ZyLXVLBvl2va0h/Y3mazzFPXmaKvM=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '8ducK1ADT1QxHYB2tGy7ce0F85v/f7e5Jh+0WoA55Exoj4ZyLXVLBvl2va0h/Y3mazzFPXmaKvM=',
   'x-amz-request-id': 'DQVF0P1VNQEFFWJJ',
   'date': 'Mon, 18 Apr 2022 16:43:32 GMT',
   'location': '/namenatbucketfinal',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'Location': '/namenatbucketfinal'}

In [6]:
def upload_file(file_name, bucket, object_name=None, args=None):
    if object_name is None:
        object_name = file_name
    response = client2.upload_file(file_name, bucket, object_name, ExtraArgs = args)
    
    print(response)
upload_file('spotname', 'yetanothercorrupteds3')

None


In [8]:
import os 
import psycopg2

ENDPOINT="database-namenat.c22yogzvdlpd.us-east-1.rds.amazonaws.com"
PORT="5432"
USR="postgres"
REGION="us-east-1"
os.environ['LIBMYSQL_ENABLE_CLEARTEXT_PLUGIN'] = '1'
session = b3.Session(profile_name='default')
client = b3.client('rds',region_name=REGION)
#token = client.generate_db_auth_token(DBHostname=ENDPOINT, Port=PORT, DBUsername=USR, Region=REGION)


conn = psycopg2.connect(host=ENDPOINT, port=PORT, user=USR, password='Leviathan2020!')

def upload_s3_to_rds(file_name, bucket, object_name=None, args=None):
    if object_name is None:
        object_name = file_name
    response = client.upload_file(file_name, bucket, object_name, ExtraArgs = args)
    
    print(response)
upload_s3_to_rds('spotname', 'namenat')

AttributeError: 'RDS' object has no attribute 'upload_file'

In [ ]:
import urllib.parse
import pyodbc

print('Loading function')

s3 = boto3.client('s3')


def lambda_handler(event, context):
    #print("Received event: " + json.dumps(event, indent=2))

    # Get the object from the event and show its content type
    bucket = event['Records'][0]['s3']['bucket']['name']
    key = urllib.parse.unquote_plus(event['Records'][0]['s3']['object']['key'], encoding='utf-8')
    try:
        response = s3.get_object(Bucket=bucket, Key=key)
    except Exception as e:
        print(e)
        print('Error getting object {} from bucket {}. Make sure they exist and your bucket is in the same region as this function.'.format(key, bucket))
        raise e
    
    # insert data from csv file into dataframe.
    # working directory for csv file: type "pwd" in Azure Data Studio or Linux
    # working directory in Windows c:\users\username
    df = pd.read_csv(response)
    # Some other example server values are
    # server = 'localhost\sqlexpress' # for a named instance
    # server = 'myserver,port' # to specify an alternate port
    server = 'database-namenat.c22yogzvdlpd.us-east-1.rds.amazonaws.com' 
    database = 'database-namenat' 
    username = 'postgres' 
    password = 'Leviathan2020!' 
    cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
    cursor = cnxn.cursor()
    # Insert Dataframe into SQL Server:
    for index, row in df.iterrows():
         cursor.execute("INSERT INTO database-namenat.natprob (name,country_code,probability) values(?,?,?)", row.name, row.country_code, row.probability)
    cnxn.commit()
    cursor.close()

In [16]:
pdfile = pd.__file__
pdfile

'C:\\Users\\ahapp\\anaconda3\\lib\\site-packages\\pandas\\__init__.py'